In [2]:
from Bio import Entrez
from Bio import SeqIO
from Bio import SearchIO
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio.SeqFeature import SeqFeature, FeatureLocation

ANÁLISE DA SEQUÊNCIA 

In [161]:
def seq_analysis(gb):
    '''
    Variáveis:
        file: nome do ficheiro com extensão ".gb"
    Returns:
        imprime as informações contidas no ficheiro
    '''

    record = SeqIO.read(gb, "genbank")
    id = record.name
    seq = record.seq
    seqlen = len(record.seq)
    source = record.annotations["source"]
    tam = len(record.annotations)
    desc = record.description
    features = len(record.features)
    totannot = record.annotations
    print(f"ID: {id} \n Sequência: {seq} \n Tamanho da sequência: {seqlen} bp \n Source: {source} \n Tamanho das anotações: {tam}")
    print(f"Descrição: {desc} \n Total features: {features}")
    print()
    print(f"Annotations: {totannot}")
    
    print()
    print("FEATURES:")
    for feat in record.features:
        print("-->", feat)
    print(f"Número de features: {features}")
    for feat in record.features:
        print("Type:", feat.type)
        print("Location:", feat.location)

    featcds = [ ]
    for i in range(len(record.features)):
        if record.features[i].type == "CDS":
            featcds.append(i)
    for k in featcds:
        print (record.features[k].location)
    for k in featcds:
        print (record.features[k].extract(record.seq))
    print(featcds)

    for feat in record.features:
        if feat.type == 'CDS':
            print("Proteína codificada: ", feat.qualifiers['product'])

    for feat in record.features:
        if feat.type == 'gene':
            print("Significado biológico: ", feat.qualifiers["note"])

    
    records = SeqIO.parse("HHEX.gb","genbank")
    count = SeqIO.write(records, "HHEX.fasta","fasta")
    print(f'Foi convertido {count} registo.')

seq_analysis("HHEX.gb")

ID: NM_002729 
 Sequência: AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGGGTGCCGCTGTACGCGCCCACGCCGCTGCTGCAACCCGCACACCCGACGCCCTTTTACATCGAGGACATCCTGGGCCGCGGGCCCGCCGCGCCCACGCCCGCCCCCACGCTGCCGTCCCCCAACTCCTCCTTCACCAGCCTCGTGTCCCCCTACCGGACCCCGGTGTACGAGCCCACGCCGATCCATCCAGCCTTCTCGCACCACTCCGCCGCCGCGCTGGCCGCTGCCTACGGACCCGGCGGCTTCGGGGGCCCTCTGTACCCCTTCCCGCGGACGGTGAACGACTACACGCACGCCCTGCTCCGCCACGACCCCCTGGGCAAACCTCTACTCTGGAGCCCCTTCTTGCAGAGGCCTCTGCATAAAAGGAAAGGCGGCCAGGTGAGATTCTCCAACGACCAGACCATCGAGCTGGAGAAGAAATTCGAGACGCAGAAATATCTCTCTCCGCCCGAGAGGAAGCGTCTGGCCAAGATGCTGCAGCTCAGCGAGAGACAGGTCAAAACCTGGTTTCAGAATCGACGCGCTAAATGGAGGAGACTAAAACAGGAGAACCCTCAAAGCAATAAAAAAGAAGAACTGGAAAGTTTGGACAGTTCCTGTGATCAGAGGCAAGATTTGCCCAGTGAACAGAATAAAGGTGCTTCTTTGGATAGCTCTCAATGTTCGCCCTCCCCTGCCTCCCAGGAAGACCTTGAATCAGAGATTTCAGAGGATTCTGATCAGGAAGTGGACATTGAGGGCGATAAAAGCTATTTTAATGCTGGATGATGACCACTGGCATTGGCATGTTCAGAAAACTGGATTTAGGAATAATGTTTTGCTACAGAAAATCTTCATAGAAGAACTGGAAGGCTATATAAGAAAGGGAATCAATTCTCTGGTATTCTGGAAACCTA

ANÁLISE DE HOMOLOGIAS POR BLAST

In [162]:

def blast(fasta_file, blast_file):
    '''
    Variáveis:
        fasta: ficheiro fasta convertido anteriormente, com extensão .fasta
        blast_file: ficheiro com o resultado do BLAST
    Returns:
        retorna um ficheiro .xml com o resultado do blastn na base de dados nucleotide
    '''
    record = SeqIO.read(fasta_file, format="fasta")
    print(len(record))
    result_handle = NCBIWWW.qblast("blastn", "nt", record.seq)
    save_file = open("blastn_HHEX.xml", "w")
    save_file.write(result_handle.read())
    save_file.close()
    result_handle.close()

    result_handle = open("blastn_HHEX.xml")
    blast_record = NCBIXML.parse(result_handle)
    for br in blast_record:
        print(f"Matrix (?): {br.matrix}")
        print(f"Database: {br.database}")
        print(f"Gap penalty: {br.gap_penalties}")   

blast("HHEX.fasta", "blastn_HHEX.xml")


1724
Matrix (?): 
Database: nt
Gap penalty: (5, 2)


In [163]:
def homologos(xml_file, evalue_thresh =None):
    '''
    Variáveis:
        file: nome do ficheiro com o resultado do BLAST
        evalue_thresh: recebe valor 0.05, por default, ou um número inteiro. Este parâmetro descreve o valor de e-value máximo aceitável para o tratamento do output do BLAST
    Returns:
        retorna uma lista dos resultados, de acordo com o valor dado pelo e-value
    '''
    evalue_tresh = 0.05
    result_handle = open(xml_file)
    blast_record = NCBIXML.parse(result_handle)
    for br in blast_record:
        print("Database: ", br.database)
        print("Gap penalty: " , br.gap_penalties)
    print(len(br.alignments))
    for br_x in br.alignments:
        print(f"Acession number: {br_x.accession}")
        print(f"ID do hit: {br_x.hit_id}")
        print(f"Definição: {br_x.hit_def}")
        print(f"HSP: {br_x.hsps}")
        break #faço break porque só quero o primeiro organismo (HUMAN) -> deve dar pra fazer algo com o entrez e selecionar o Homo sapiens
    print()
    for alignment in br.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < evalue_tresh:
                print("        ***ALINHAMENTO***")
                print(f"Identidade: {hsp.identities}")
                print(f"E-value: {hsp.expect}")
                print(f"Score: {hsp.score}")
                print(f"Tamanho: {hsp.align_length}")
                print(f"Caracteres iguais: {len(hsp.match)}")
                print("Query " + hsp.query[0:90] + "...")
                print("Match " + hsp.match[0:90] + "...")
                print("Sbjct " + hsp.sbjct[0:90] + "...")
                print()
    blastq_result = SearchIO.read("blastn_HHEX.xml", "blast-xml")
    print(blastq_result)
    
    result_handle.close()
    
    blast_slice = blastq_result[:10]
    print(blast_slice)
    
homologos("blastn_HHEX.xml")

Database:  nt
Gap penalty:  (5, 2)
50
Acession number: NM_002729
ID do hit: gi|1519245767|ref|NM_002729.5|
Definição: Homo sapiens hematopoietically expressed homeobox (HHEX), mRNA
HSP: [<Bio.Blast.Record.HSP object at 0x000002371149A590>]

        ***ALINHAMENTO***
Identidade: 1724
E-value: 0.0
Score: 3448.0
Tamanho: 1724
Caracteres iguais: 1724
Query AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGGGTGCCGCTGTACGCGC...
Match ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
Sbjct AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGGGTGCCGCTGTACGCGC...

        ***ALINHAMENTO***
Identidade: 1723
E-value: 0.0
Score: 3446.0
Tamanho: 1723
Caracteres iguais: 1723
Query AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGGGTGCCGCTGTACGCGC...
Match ||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||...
Sbjct AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGC

In [164]:
def HHEX(gb, blast_file, blast = False, evalue_tresh = 0.05):
    '''
    Variáveis:
        gb: ficheiro genbank obtido online (HHEX.gb), com extensão .gb
        blast_file:  nome do ficheiro com os resultados do BLAST ("blastn_HHEX.xml") #depois cada um alterar para o nome do seu gene
        blast: recebe o valor False por default. Se blast = True, realiza um BLAST.
        evalue_tresh:  recebe valor 0.05, por default, ou um número inteiro. Este parâmetro descreve o valor de e-value máximo aceitável para o tratamento do output do BLAST
    Returns:
        gera um ficheiro em formato de texto (.txt), que contém os Acession Number dos resultados do BLAST segundo o valor de e-value
        '''

    seq_analysis(gb)
    if blast == True:
        blast(gb, blast_file)
    homologos(blast_file, evalue_thresh =0.05)
    #falta gerar ficheiro txt

HHEX("HHEX.gb", "blastn_HHEX.xml", blast, 0.05)

#esta def resume todas as outras acima, por isso meio que não +e necessário chamar as outras no final de cada

ID: NM_002729 
 Sequência: AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGGGTGCCGCTGTACGCGCCCACGCCGCTGCTGCAACCCGCACACCCGACGCCCTTTTACATCGAGGACATCCTGGGCCGCGGGCCCGCCGCGCCCACGCCCGCCCCCACGCTGCCGTCCCCCAACTCCTCCTTCACCAGCCTCGTGTCCCCCTACCGGACCCCGGTGTACGAGCCCACGCCGATCCATCCAGCCTTCTCGCACCACTCCGCCGCCGCGCTGGCCGCTGCCTACGGACCCGGCGGCTTCGGGGGCCCTCTGTACCCCTTCCCGCGGACGGTGAACGACTACACGCACGCCCTGCTCCGCCACGACCCCCTGGGCAAACCTCTACTCTGGAGCCCCTTCTTGCAGAGGCCTCTGCATAAAAGGAAAGGCGGCCAGGTGAGATTCTCCAACGACCAGACCATCGAGCTGGAGAAGAAATTCGAGACGCAGAAATATCTCTCTCCGCCCGAGAGGAAGCGTCTGGCCAAGATGCTGCAGCTCAGCGAGAGACAGGTCAAAACCTGGTTTCAGAATCGACGCGCTAAATGGAGGAGACTAAAACAGGAGAACCCTCAAAGCAATAAAAAAGAAGAACTGGAAAGTTTGGACAGTTCCTGTGATCAGAGGCAAGATTTGCCCAGTGAACAGAATAAAGGTGCTTCTTTGGATAGCTCTCAATGTTCGCCCTCCCCTGCCTCCCAGGAAGACCTTGAATCAGAGATTTCAGAGGATTCTGATCAGGAAGTGGACATTGAGGGCGATAAAAGCTATTTTAATGCTGGATGATGACCACTGGCATTGGCATGTTCAGAAAACTGGATTTAGGAATAATGTTTTGCTACAGAAAATCTTCATAGAAGAACTGGAAGGCTATATAAGAAAGGGAATCAATTCTCTGGTATTCTGGAAACCTA

FERRAMENTAS DE ANÁLISE DE PROTEÍNAS

In [165]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from Bio import SearchIO
from Bio import ExPASy

In [168]:
def get_protein(id):
    """
    Variávies:
        id: identificador da proteína obtido pela swissprot
    Returns:
    """
    handle = ExPASy.get_sprot_raw("Q03014")
    seq_record = SeqIO.read(handle, "swiss")
    id = seq_record.id
    seq = seq_record.seq
    tam = len(seq_record.seq)
    name = seq_record.name
    desc = seq_record.description
    com = seq_record.annotations["comment"]
    taxon = seq_record.annotations["taxonomy"]
    organism = seq_record.annotations["organism"]
    key = seq_record.annotations["keywords"]
    print(f"ID {id} \n Sequência: {seq} \n Tamanho da sequência: {tam} aa")
    print(f"Nome: {name} \n Descrição: {desc} \n Taxonomia: {taxon} \n Organismo: {organism} \n Keywords: {key}")
    
get_protein(id)

ID Q03014 
 Sequência: MQYPHPGPAAGAVGVPLYAPTPLLQPAHPTPFYIEDILGRGPAAPTPAPTLPSPNSSFTSLVSPYRTPVYEPTPIHPAFSHHSAAALAAAYGPGGFGGPLYPFPRTVNDYTHALLRHDPLGKPLLWSPFLQRPLHKRKGGQVRFSNDQTIELEKKFETQKYLSPPERKRLAKMLQLSERQVKTWFQNRRAKWRRLKQENPQSNKKEELESLDSSCDQRQDLPSEQNKGASLDSSQCSPSPASQEDLESEISEDSDQEVDIEGDKSYFNAG 
 Tamanho da sequência: 270 aa
Nome: HHEX_HUMAN 
 Descrição: RecName: Full=Hematopoietically-expressed homeobox protein HHEX; Short=Homeobox protein HEX; AltName: Full=Homeobox protein PRH; 
 Taxonomia: ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo'] 
 Organismo: Homo sapiens (Human) 
 Keywords: ['3D-structure', 'Developmental protein', 'Differentiation', 'DNA-binding', 'Homeobox', 'Nucleus', 'Phosphoprotein', 'Reference proteome', 'Repressor', 'Transcription', 'Transcription regulation', 'Wnt signaling pathway']


In [169]:
handle = ExPASy.get_sprot_raw("Q03014")
seq_record = SeqIO.read(handle, "swiss")
seq = seq_record.seq

In [170]:
def prot_blast(blastp_file, query):
    """ 
    Variáveis:
        blastp_file: ficheiro com o resultado do blastp (ex: prot_blastp_nr.xml
        query: sequência proteica query
    Returns:
        retorna um ficheiro .xml com o resultado do blastn na base de dados (swissprot?)
    """
    seq_prot= query
    result_handle = NCBIWWW.qblast('blastp', 'nr', seq_prot)

    save_file = open("prot_blastp_nr.xml","w")
    save_file.write(result_handle.read())
    save_file.close()

    blast_record = NCBIXML.parse(result_handle)
    result_handle.close()

prot_blast("prot_blastp_nr.xml", seq)

#aqui fiz o blast na DB non-redundant pra descobrir os homólogos,
#depois vou fazer na swiss prot. Não sei se faz sentido

In [171]:
def homologos_p (blastp_file, evalue_tresh = None):
    """
    Variáveis:
        blastp_file:
        evalue_tresh:
    Returns:
        
    """
    result_handle = open(blastp_file)
    blast_record = NCBIXML.read(result_handle)
    FILE = str("seqshomologas_blastp.fasta")
    save_file = open(FILE, 'w+')
    if evalue_tresh == None:
        evalue_tresh = 0.05
    for alignment in blast_record.alignments:
        for hsp in range(len(alignment.hsps)):
            if alignment.hsps[hsp].expect < evalue_tresh:
                if hsp != 0:
                    pass
                else:
                    save_file.write('>' + alignment.title + '\n' + alignment.hsps[hsp].sbjct + '\n')
    
    result_handle.close()

homologos_p("prot_blastp_nr.xml")

In [172]:
def prot_file(blastp_file):
    """
    Variáveis:
        blast_file:
    Returns:
        
    """
    result_handle = open(blastp_file)
    blast_record = NCBIXML.read(result_handle)
    evalue_tresh = 1e-30
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            if hsp.expect < evalue_tresh:
                print("      ***ALINHAMENTO***")
                print("Sequência: ", alignment.title)
                print("Tamanho da sequência: ", alignment.length)
                print("E-value:", hsp.expect)
                print("Score: ", hsp.score)
                print(f"Caracteres iguais: {len(hsp.match)}")
                print(hsp.query[0:75] + "...")
                print(hsp.match[0:75] + "...")
                print(hsp.sbjct[0:75] + "...")
                print()
    
    blastq_result = SearchIO.read(blastp_file, "blast-xml")
    print(blastq_result)
    for br in blastq_result:
        print(f'Sequence ID: {br.id}')
        print(f'Description: {br.description}')
        print(f'E-value: {br[0].evalue}')
        print(f'Bit Score: {br[0].bitscore}')
        print(f'Alignment:\n{br[0].aln}')
        print()
    result_handle.close()

prot_file("prot_blastp_nr.xml")

      ***ALINHAMENTO***
Sequência:  ref|NP_002720.1| hematopoietically-expressed homeobox protein HHEX [Homo sapiens] >ref|XP_003825722.2| hematopoietically-expressed homeobox protein HHEX [Pan paniscus] >ref|XP_507925.2| hematopoietically-expressed homeobox protein HHEX [Pan troglodytes] >sp|Q03014.1| RecName: Full=Hematopoietically-expressed homeobox protein HHEX; Short=Homeobox protein HEX; AltName: Full=Homeobox protein PRH [Homo sapiens] >gb|ABZ92007.1| hematopoietically expressed homeobox, partial [synthetic construct] >gb|AAH15110.1| Hematopoietically expressed homeobox [Homo sapiens] >gb|AAH50638.1| Hematopoietically expressed homeobox [Homo sapiens] >gb|EAW50087.1| homeobox, hematopoietically expressed, isoform CRA_a [Homo sapiens] >gb|EAW50088.1| homeobox, hematopoietically expressed, isoform CRA_a [Homo sapiens]
Tamanho da sequência:  270
E-value: 0.0
Score:  1422.0
Caracteres iguais: 270
MQYPHPGPAAGAVGVPLYAPTPLLQPAHPTPFYIEDILGRGPAAPTPAPTLPSPNSSFTSLVSPYRTPVYEPTPI...
MQYPHPGP

In [173]:
def PROTEIN(id, blastp_file, blast = False, evalue_tresh = None):
    """ 
    Variáveis:
    
    Returns:
    
    """
    if blast == True:
        from Bio import ExPASy
        with ExPASy.get_sprot_raw(id) as handle:
            seq_record = SeqIO.read(handle, "swiss")
        prot_blast(blastp_file, seq_record.seq)
    x = get_protein(id)
    print(x)
    homologos_p(blastp_file, evalue_tresh)
    prot_file(blastp_file)
PROTEIN("Q03014", "prot_blastp_swiss.xml", False)

ID Q03014 
 Sequência: MQYPHPGPAAGAVGVPLYAPTPLLQPAHPTPFYIEDILGRGPAAPTPAPTLPSPNSSFTSLVSPYRTPVYEPTPIHPAFSHHSAAALAAAYGPGGFGGPLYPFPRTVNDYTHALLRHDPLGKPLLWSPFLQRPLHKRKGGQVRFSNDQTIELEKKFETQKYLSPPERKRLAKMLQLSERQVKTWFQNRRAKWRRLKQENPQSNKKEELESLDSSCDQRQDLPSEQNKGASLDSSQCSPSPASQEDLESEISEDSDQEVDIEGDKSYFNAG 
 Tamanho da sequência: 270 aa
Nome: HHEX_HUMAN 
 Descrição: RecName: Full=Hematopoietically-expressed homeobox protein HHEX; Short=Homeobox protein HEX; AltName: Full=Homeobox protein PRH; 
 Taxonomia: ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo'] 
 Organismo: Homo sapiens (Human) 
 Keywords: ['3D-structure', 'Developmental protein', 'Differentiation', 'DNA-binding', 'Homeobox', 'Nucleus', 'Phosphoprotein', 'Reference proteome', 'Repressor', 'Transcription', 'Transcription regulation', 'Wnt signaling pathway']
None
      ***ALINHAMENTO***
Sequência:  sp|Q03014

In [174]:
from Bio.PDB.PDBParser import PDBParser

In [175]:
def PDB(id, pdb_file):
    """ 
    Variáveis:
    
    Returns:
    """
    p = PDBParser(PERMISSIVE=1)
    s = p.get_structure(id, pdb_file)
    for chain in s[0]:
        print(f'Chain ID: {chain.id}')
    smeth = s.header['structure_method']
    keywords = s.header['keywords']
    comp = s.header["compound"]
    print("Keywords: " , keywords)
    print("Structure Method: ", smeth)
    print("Composto: ", comp)

    import nglview as nv
    nv.show_biopython(s, gui=True)
    #isto funciona no jupyter notebook mas aqui dá erro... mas é isto

PDB("2E10", "2e1o.pdb")

Chain ID: A
Keywords:  dna binding protein, structural genomics, nppsfa, national project on protein structural and functional analyses, riken structural genomics/proteomics initiative, rsgi, unknown function
Structure Method:  solution nmr
Composto:  {'1': {'misc': '', 'molecule': 'homeobox protein prh', 'chain': 'a', 'fragment': 'homeobox domain', 'synonym': 'hematopoietically expressed homeobox, homeobox protein hex', 'engineered': 'yes'}}


AttributeError: 'super' object has no attribute '_ipython_display_'

In [120]:
def CDD(cdd_file):
    result_handle = NCBIWWW.qblast("blastp", "CDD", seq)
    result_handle = open(cdd_file)
    blast_records = SearchIO.read(result_handle, "blast-xml")
    print(blast_records[:])
    result_handle.close()
CDD("cdd_file")


KeyboardInterrupt: 

Alinhamento Múltiplo

In [5]:
from Bio.Align import MultipleSeqAlignment
from Bio.Blast import NCBIXML
from Bio.Blast import NCBIWWW
from Bio import SeqIO
from Bio import AlignIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Align import AlignInfo

In [3]:
def align(blast_file, align_file):
    """
    Variáveis:
        blast_file:
        align_file:
    Returns:
    """
    result_blast = open(blast_file)
    blast_records = NCBIXML.read(result_blast)
    FILE = str("obterseqs.fasta")
    save_file = open(FILE, 'w+')
    for alignment in blast_records.alignments:
        for hsp in alignment.hsps:
            save_file.write('>' + alignment.title + '\n' + hsp.sbjct[0:45] + '\n')
            print(">", alignment.title, "\n", hsp.query)
            print()
align("blastn_HHEX.xml", "obterseqs.fasta")

> gi|1519245767|ref|NM_002729.5| Homo sapiens hematopoietically expressed homeobox (HHEX), mRNA 
 AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGGGTGCCGCTGTACGCGCCCACGCCGCTGCTGCAACCCGCACACCCGACGCCCTTTTACATCGAGGACATCCTGGGCCGCGGGCCCGCCGCGCCCACGCCCGCCCCCACGCTGCCGTCCCCCAACTCCTCCTTCACCAGCCTCGTGTCCCCCTACCGGACCCCGGTGTACGAGCCCACGCCGATCCATCCAGCCTTCTCGCACCACTCCGCCGCCGCGCTGGCCGCTGCCTACGGACCCGGCGGCTTCGGGGGCCCTCTGTACCCCTTCCCGCGGACGGTGAACGACTACACGCACGCCCTGCTCCGCCACGACCCCCTGGGCAAACCTCTACTCTGGAGCCCCTTCTTGCAGAGGCCTCTGCATAAAAGGAAAGGCGGCCAGGTGAGATTCTCCAACGACCAGACCATCGAGCTGGAGAAGAAATTCGAGACGCAGAAATATCTCTCTCCGCCCGAGAGGAAGCGTCTGGCCAAGATGCTGCAGCTCAGCGAGAGACAGGTCAAAACCTGGTTTCAGAATCGACGCGCTAAATGGAGGAGACTAAAACAGGAGAACCCTCAAAGCAATAAAAAAGAAGAACTGGAAAGTTTGGACAGTTCCTGTGATCAGAGGCAAGATTTGCCCAGTGAACAGAATAAAGGTGCTTCTTTGGATAGCTCTCAATGTTCGCCCTCCCCTGCCTCCCAGGAAGACCTTGAATCAGAGATTTCAGAGGATTCTGATCAGGAAGTGGACATTGAGGGCGATAAAAGCTATTTTAATGCTGGATGATGACCACTGGCATTGGCATGTTCAGAAAACTGGATTTAGGAATAATGTTTTGCTACA

In [7]:
def parse_align(align_file):
    alignments = AlignIO.parse(align_file,format = "fasta")
    for alignment in alignments:
        print(alignment)
    AlignIO.write(alignment, "align_HHEX", "fasta")
parse_align("obterseqs.fasta")

Alignment with 54 rows and 45 columns
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1519245767|ref|NM_002729.5|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|15929354|gb|BC015110.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|15680040|gb|BC014336.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1367123331|ref|XM_507925.5|
CGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGT gi|32547|emb|X67235.1|
GCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGG gi|30048158|gb|BC050638.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1849002271|ref|XM_003825674.3|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1753031712|ref|XM_031015733.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1351383818|ref|XM_024253783.1|
GCGAGGGGCGGG--CGCGGCGGAGCCATGCAGTACCCGCACCCCG gi|292404|gb|L16499.1|HUMPRH
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTATCCGC gi|1743203112|ref|XM_030802514.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1059150898|ref|XM_017876753.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC

In [8]:
def stockholm(align_file, stock_file):
    """
    Variáveis:

    Returns:
    """
    form_stock = AlignIO.parse(align_file,"fasta")
    AlignIO.convert(align_file,"fasta", stock_file,"stockholm")
    #dá erro de ids duplicados
stockholm("align_HHEX", "align_results_HHEX.sth")

ValueError: Duplicate record identifier: gi|1799962366|ref|XM_032170111.1|

In [20]:
alignments = AlignIO.parse("align_HHEX",format = "fasta")
summary_align = AlignInfo.SummaryInfo(alignment)
consensus = summary_align.dumb_consensus()
consensus

Seq('AGCTCTXCGAGGGGCCGGAGCGCXGCGGAGCCATGCAGTACCCGC')

Árvore filogenética

In [182]:
from Bio import Phylo
from Bio import AlignIO

In [183]:
alignment = AlignIO.read(open("align_results_hhex.sth"),"stockholm")
print(alignment)

Alignment with 30 rows and 45 columns
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1519245767|ref|NM_002729.5|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|15929354|gb|BC015110.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|15680040|gb|BC014336.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1367123331|ref|XM_507925.5|
CGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGT gi|32547|emb|X67235.1|
GCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCGTGGGG gi|30048158|gb|BC050638.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1849002271|ref|XM_003825674.3|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1753031712|ref|XM_031015733.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1351383818|ref|XM_024253783.1|
GCGAGGGGCGGG--CGCGGCGGAGCCATGCAGTACCCGCACCCCG gi|292404|gb|L16499.1|HUMPRH
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTATCCGC gi|1743203112|ref|XM_030802514.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC gi|1059150898|ref|XM_017876753.1|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTACCCGC

In [150]:
from Bio.Phylo.TreeConstruction import DistanceCalculator
from Bio import AlignIO

calculator = DistanceCalculator('blosum62')
dm = calculator.get_distance(alignment)
print(dm)

gi|1519245767|ref|NM_002729.5|	0
gi|15929354|gb|BC015110.1|	0.0	0
gi|15680040|gb|BC014336.1|	0.0	0.0	0
gi|1367123331|ref|XM_507925.5|	0.0	0.0	0.0	0
gi|32547|emb|X67235.1|	0.956989247311828	0.956989247311828	0.956989247311828	0.956989247311828	0
gi|30048158|gb|BC050638.1|	0.8098591549295775	0.8098591549295775	0.8098591549295775	0.8098591549295775	0.8169014084507042	0
gi|1849002271|ref|XM_003825674.3|	0.0	0.0	0.0	0.0	0.956989247311828	0.8098591549295775	0
gi|1753031712|ref|XM_031015733.1|	0.0	0.0	0.0	0.0	0.956989247311828	0.8098591549295775	0.0	0
gi|1351383818|ref|XM_024253783.1|	0.0	0.0	0.0	0.0	0.956989247311828	0.8098591549295775	0.0	0.0	0
gi|292404|gb|L16499.1|HUMPRH	0.884	0.884	0.884	0.884	0.8544776119402985	0.8909774436090225	0.884	0.884	0.884	0
gi|1743203112|ref|XM_030802514.1|	0.0	0.0	0.0	0.0	0.956989247311828	0.8098591549295775	0.0	0.0	0.0	0.884	0
gi|1059150898|ref|XM_017876753.1|	0.0	0.0	0.0	0.0	0.956989247311828	0.8098591549295775	0.0	0.0	0.0	0.884	0.0	0
gi|795247561|ref|XM_012

In [151]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor
constructor = DistanceTreeConstructor()
upgmatree = constructor.upgma(dm)
print(upgmatree)

Tree(rooted=True)
    Clade(branch_length=0, name='Inner29')
        Clade(name='gi|292404|gb|L16499.1|HUMPRH')
        Clade(name='Inner28')
            Clade(name='Inner27')
                Clade(name='Inner25')
                    Clade(name='gi|795196478|ref|XM_011947898.1|')
                    Clade(name='gi|30048158|gb|BC050638.1|')
                Clade(name='gi|32547|emb|X67235.1|')
            Clade(name='Inner26')
                Clade(name='gi|1799962366|ref|XM_032170111.1|')
                Clade(name='Inner24')
                    Clade(name='Inner22')
                        Clade(name='Inner19')
                            Clade(name='gi|1933975555|ref|XM_017498186.2|')
                            Clade(name='gi|1804417576|ref|XM_032245527.1|')
                        Clade(name='gi|1864419579|ref|XM_035268475.1|')
                    Clade(name='Inner23')
                        Clade(name='gi|817280149|ref|XM_012454782.1|')
                        Clade(name='Inner21'

In [154]:
form_phyl = AlignIO.parse("align_results_hhex.sth","stockholm")
AlignIO.convert("align_results_hhex.sth","stockholm","align_results_hhex.phy","phylip")

alignments = AlignIO.parse("align_results_hhex.phy", "phylip")
for alignment in alignments:
    print(alignment)
    print()

Alignment with 30 rows and 40 columns
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1519245
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1592935
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1568004
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1367123
CGGAGCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGC gi|32547|e
GCCATGCAGTACCCGCACCCCGGGCCGGCGGCGGGCGCCG gi|3004815
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1849002
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1753031
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1351383
GCGAGGGGCGGG--CGCGGCGGAGCCATGCAGTACCCGCA gi|292404|
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1743203
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1059150
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|7952475
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1825822
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1938880
AGCTCTGCGAGGGGCCGAAGCGCGGCGGAGCCATGCAGTA gi|1411179
AGCTCTGCGAGGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|1751199
AGCTCTGCGAAGGGCCGGAGCGCGGCGGAGCCATGCAGTA gi|2161943
...
GGCAAACCTCTACTCTGGAGCC

In [155]:
njtree = constructor.nj(dm)
print(njtree)

Tree(rooted=False)
    Clade(branch_length=0, name='Inner28')
        Clade(name='Inner27')
            Clade(name='Inner26')
                Clade(name='Inner25')
                    Clade(name='gi|795247561|ref|XM_012063055.1|')
                    Clade(name='Inner24')
                        Clade(name='gi|1059150898|ref|XM_017876753.1|')
                        Clade(name='Inner23')
                            Clade(name='gi|1743203112|ref|XM_030802514.1|')
                            Clade(name='Inner22')
                                Clade(name='gi|1351383818|ref|XM_024253783.1|')
                                Clade(name='Inner21')
                                    Clade(name='gi|1753031712|ref|XM_031015733.1|')
                                    Clade(name='Inner20')
                                        Clade(name='gi|1849002271|ref|XM_003825674.3|')
                                        Clade(name='Inner19')
                                            Clade(name='I

In [156]:
Phylo.write([upgmatree, njtree], "phylo_trees.nhx","newick")

2

In [157]:
tree_up = Phylo.draw_ascii(upgmatree)

  ____________________________________ gi|292404|gb|L16499.1|HUMPRH
 |
 |            ___________________________ gi|795196478|ref|XM_011947898.1|
 |       ____|
_|    __|    |___________________________ gi|30048158|gb|BC050638.1|
 |   |  |
 |   |  |________________________________ gi|32547|emb|X67235.1|
 |   |
 |___|     ______________________________ gi|1799962366|ref|XM_032170111.1|
     |    |
     |    |                             , gi|1933975555|ref|XM_017498186.2|
     |    |                            ,|
     |____|                           ,|| gi|1804417576|ref|XM_032245527.1|
          |                           ||
          |                           ||_ gi|1864419579|ref|XM_035268475.1|
          |                           |
          |___________________________|__ gi|817280149|ref|XM_012454782.1|
                                      |
                                      |  , gi|1777257527|ref|XM_003904001.5|
                                      | ,|
              

In [158]:
tree_nj = Phylo.draw_ascii(njtree)

 , gi|795247561|ref|XM_012063055.1|
 |
 , gi|1059150898|ref|XM_017876753.1|
 |
 , gi|1743203112|ref|XM_030802514.1|
 |
 , gi|1351383818|ref|XM_024253783.1|
 |
 , gi|1753031712|ref|XM_031015733.1|
 |
 , gi|1849002271|ref|XM_003825674.3|
 |
 , gi|1519245767|ref|NM_002729.5|
 |
 | gi|15929354|gb|BC015110.1|
 |
 | gi|15680040|gb|BC014336.1|
 |
 | gi|1367123331|ref|XM_507925.5|
 |
 |, gi|1777257527|ref|XM_003904001.5|
 ,|
 || gi|1411179766|ref|XM_025397020.1|
 |
 |, gi|1622966527|ref|XM_001090715.4|
 ,|
 |, gi|795332012|ref|XM_011738821.1|
 ||
 |, gi|2309500272|ref|XM_050802826.1|
 ||
 || gi|2161943361|ref|XM_005565955.3|
 |
 , gi|1788697486|ref|XM_023226304.3|
 |
 , gi|1984105766|ref|XM_003922390.3|
 |
 |, gi|817280149|ref|XM_012454782.1|
 ||
 ||, gi|1804417576|ref|XM_032245527.1|
 |,|
 ||, gi|1933975555|ref|XM_017498186.2|
 |||
 ||| gi|1864419579|ref|XM_035268475.1|
 ||
 ||             _________________ gi|1799962366|ref|XM_032170111.1|
_||            |
 ||____________|     ____________ g